In [1]:
import os 
os.chdir("../")


In [2]:
print("ahmad")

ahmad


In [3]:
%pwd

'c:\\Users\\aamer\\OneDrive\\Desktop\\Projects\\train2'

In [4]:
import CNNproject

In [11]:
import sys
print(sys.path)


['c:\\Users\\aamer\\miniconda3\\envs\\myenv\\python39.zip', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\DLLs', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\lib', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv', '', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\lib\\site-packages', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\lib\\site-packages\\win32', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\aamer\\miniconda3\\envs\\myenv\\lib\\site-packages\\Pythonwin']


In [ ]:
import tensorflow as tf 

is_augmentaion =False

train_data_dir = "artifacts/data_ingestion/Chicken-fecal-images"

datagenerator_kwargs = dict(rescale=1./255, validation_split=0.20)

dataflow_kwargs = dict(
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Process images in batches of 32
    interpolation="bilinear"
)

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs) #preprocessing

valid_generator = valid_datagenerator.flow_from_directory(
    directory=train_data_dir,
    subset="validation",
    shuffle=False,
    **dataflow_kwargs
)

if  is_augmentaion:
  train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
  rotation_range=40,
  horizontal_flip=True,
  width_shift_range=0.2,
  height_shift_range=0.2,
  **datagenerator_kwargs)

else : 
  train_datagenerator = valid_datagenerator
# Display batch shape

train_generator = train_datagenerator.flow_from_directory(
    directory=train_data_dir,
    subset="training",
    shuffle=True,
    **dataflow_kwargs
)


for batch_images, batch_labels in valid_generator:
    print("Batch image shape:", batch_images.shape)
    print("Batch label shape:", batch_labels.shape)
    print(valid_generator.class_indices)
    break  # Print for only one batch

print("*******")
for batch_images, batch_labels in train_generator:
    print("Batch image shape:", batch_images.shape)
    print("Batch label shape:", batch_labels.shape)
    break  # Print for only one batc


model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # 2 classes
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 10
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
callback_list = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)]


history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    callbacks=callback_list
)